# Reaction Emoji Clustering Analysis

「同じリアクション絵文字は似たようなコンテキスト（感情・状況）で使われる」という仮説を検証するための分析ノートブック。

## 概要

このノートブックでは以下を行います：

1. **データの読み込み**: クラスタリング結果とリアクションコンテキストを読み込み
2. **2D可視化**: UMAP/t-SNEによる次元削減と散布図
3. **クラスタ分析**: 各クラスタの構成と代表的なメッセージの確認
4. **ユーザー行動パターン**: ヒートマップによる可視化

In [ ]:
import sys
sys.path.insert(0, '../src')

import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from slack_graph.storage import Storage
from slack_graph.clustering.features import TextFeatureExtractor, BehaviorFeatureExtractor, FeatureCombiner
from slack_graph.clustering.cluster import ReactionClusterer, KMeansClusterer

## 1. データの読み込み

In [ ]:
# DBに接続（テストDBまたは本番DBを指定）
DB_PATH = '../data/test_reactions.db'  # テストデータ
# DB_PATH = '../data/slack_graph.db'  # 本番データ

store = Storage(DB_PATH)
store.init()

# リアクションコンテキストを構築
context_count = store.build_reaction_contexts()
print(f"Reaction contexts: {context_count}")
print(f"Unique reactions: {len(store.get_unique_reactions())}")

In [ ]:
# リアクションコンテキストをDataFrameに変換
contexts = list(store.get_reaction_contexts())
df_contexts = pd.DataFrame(contexts)
df_contexts.head()

In [ ]:
# リアクション別の統計
reaction_stats = df_contexts.groupby('reaction_name').agg({
    'message_ts': 'count',
    'reactor_user': 'nunique'
}).rename(columns={'message_ts': 'count', 'reactor_user': 'unique_users'})
reaction_stats = reaction_stats.sort_values('count', ascending=False)
reaction_stats

## 2. 特徴抽出

In [ ]:
# テキスト特徴量を抽出
text_extractor = TextFeatureExtractor(model_name='paraphrase-multilingual-MiniLM-L12-v2')
text_features, reaction_names = text_extractor.get_reaction_embeddings(store)
print(f"Text features shape: {text_features.shape}")
print(f"Reactions: {reaction_names}")

In [ ]:
# 行動特徴量を抽出
behavior_extractor = BehaviorFeatureExtractor()
behavior_features = behavior_extractor.get_behavior_features(store, reaction_names)
print(f"Behavior features shape: {behavior_features.shape}")

In [ ]:
# 特徴量を結合
combiner = FeatureCombiner(text_weight=0.5, behavior_weight=0.5)
combined_features = combiner.combine(text_features, behavior_features)
print(f"Combined features shape: {combined_features.shape}")

## 3. クラスタリング

In [ ]:
# K-Meansでクラスタリング（クラスタ数を指定）
kmeans_clusterer = KMeansClusterer(n_clusters=6)
kmeans_result = kmeans_clusterer.fit(combined_features, reaction_names)

print(f"K-Means: {kmeans_result.n_clusters} clusters")
print(f"Silhouette score: {kmeans_result.silhouette_score:.3f}")
print("\nCluster assignments:")
for cluster_id, members in kmeans_result.get_clusters_summary().items():
    print(f"  Cluster {cluster_id}: {', '.join(members)}")

In [ ]:
# HDBSCANでクラスタリング（自動クラスタ数決定）
hdbscan_clusterer = ReactionClusterer(min_cluster_size=2, min_samples=1)
hdbscan_result = hdbscan_clusterer.fit(combined_features, reaction_names)

print(f"HDBSCAN: {hdbscan_result.n_clusters} clusters")
if hdbscan_result.silhouette_score:
    print(f"Silhouette score: {hdbscan_result.silhouette_score:.3f}")
print("\nCluster assignments:")
for cluster_id, members in hdbscan_result.get_clusters_summary().items():
    label = "Noise" if cluster_id == -1 else f"Cluster {cluster_id}"
    print(f"  {label}: {', '.join(members)}")

## 4. 2D可視化（UMAP）

In [ ]:
from umap import UMAP
from sklearn.preprocessing import StandardScaler

# 特徴量を標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(combined_features)

# UMAPで2次元に削減
reducer = UMAP(n_components=2, random_state=42, n_neighbors=min(15, len(reaction_names)-1))
coords_2d = reducer.fit_transform(scaled_features)

print(f"UMAP output shape: {coords_2d.shape}")

In [ ]:
# DataFrameを作成
df_viz = pd.DataFrame({
    'reaction': reaction_names,
    'x': coords_2d[:, 0],
    'y': coords_2d[:, 1],
    'kmeans_cluster': [str(c) for c in kmeans_result.labels],
    'hdbscan_cluster': [str(c) for c in hdbscan_result.labels],
    'count': [reaction_stats.loc[r, 'count'] if r in reaction_stats.index else 0 for r in reaction_names]
})

df_viz

In [ ]:
# K-Meansクラスタの散布図
fig = px.scatter(
    df_viz,
    x='x',
    y='y',
    color='kmeans_cluster',
    text='reaction',
    size='count',
    title='Reaction Emoji Clusters (K-Means)',
    labels={'x': 'UMAP 1', 'y': 'UMAP 2', 'kmeans_cluster': 'Cluster'},
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textposition='top center')
fig.update_layout(height=600, width=800)
fig.show()

In [ ]:
# HDBSCANクラスタの散布図
fig = px.scatter(
    df_viz,
    x='x',
    y='y',
    color='hdbscan_cluster',
    text='reaction',
    size='count',
    title='Reaction Emoji Clusters (HDBSCAN)',
    labels={'x': 'UMAP 1', 'y': 'UMAP 2', 'hdbscan_cluster': 'Cluster'},
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textposition='top center')
fig.update_layout(height=600, width=800)
fig.show()

## 5. ユーザー×リアクション ヒートマップ

In [ ]:
# ユーザー×リアクションのカウントを取得
user_reaction_counts = store.get_user_reaction_counts()

# DataFrameに変換
users = sorted(user_reaction_counts.keys())
reactions = sorted(set(r for counts in user_reaction_counts.values() for r in counts.keys()))

matrix = np.zeros((len(users), len(reactions)))
for i, user in enumerate(users):
    for j, reaction in enumerate(reactions):
        matrix[i, j] = user_reaction_counts.get(user, {}).get(reaction, 0)

df_heatmap = pd.DataFrame(matrix, index=users, columns=reactions)
df_heatmap

In [ ]:
# ヒートマップを描画
fig = px.imshow(
    df_heatmap,
    labels=dict(x="Reaction", y="User", color="Count"),
    title="User-Reaction Usage Heatmap",
    color_continuous_scale="YlOrRd",
    aspect="auto"
)
fig.update_layout(height=400, width=1000)
fig.show()

## 6. クラスタ別の代表メッセージ

In [ ]:
# 各クラスタの代表的なメッセージを表示
cluster_assignments = dict(zip(reaction_names, kmeans_result.labels))

for cluster_id in sorted(set(kmeans_result.labels)):
    print(f"\n{'='*60}")
    print(f"Cluster {cluster_id}")
    print('='*60)
    
    # このクラスタに属するリアクション
    cluster_reactions = [r for r, c in cluster_assignments.items() if c == cluster_id]
    print(f"Reactions: {', '.join(cluster_reactions)}")
    print()
    
    # 代表的なメッセージ（各リアクションから1つずつ）
    print("Sample messages:")
    for reaction in cluster_reactions[:3]:  # 最大3つのリアクションのみ
        messages = store.get_messages_for_reaction(reaction)
        if messages:
            sample = messages[0][:100]  # 最初の100文字
            print(f"  :{reaction}: → \"{sample}...\"")

## 7. リアクション共起分析

In [ ]:
# リアクション共起行列を取得
cooccurrence = store.get_reaction_cooccurrence()

# 行列に変換
co_matrix = np.zeros((len(reactions), len(reactions)))
for i, r1 in enumerate(reactions):
    for j, r2 in enumerate(reactions):
        if r1 in cooccurrence and r2 in cooccurrence[r1]:
            co_matrix[i, j] = cooccurrence[r1][r2]

df_cooccurrence = pd.DataFrame(co_matrix, index=reactions, columns=reactions)

In [ ]:
# 共起ヒートマップ
fig = px.imshow(
    df_cooccurrence,
    labels=dict(x="Reaction", y="Reaction", color="Co-occurrence"),
    title="Reaction Co-occurrence Matrix",
    color_continuous_scale="Blues",
    aspect="equal"
)
fig.update_layout(height=700, width=800)
fig.show()

## 8. パラメータ感度分析

In [ ]:
# テキスト重みを変えてクラスタリング結果がどう変わるか
results = []

for text_weight in [0.0, 0.25, 0.5, 0.75, 1.0]:
    behavior_weight = 1.0 - text_weight
    combiner = FeatureCombiner(text_weight=max(text_weight, 0.01), behavior_weight=max(behavior_weight, 0.01))
    combined = combiner.combine(text_features, behavior_features)
    
    clusterer = KMeansClusterer(n_clusters=6)
    result = clusterer.fit(combined, reaction_names)
    
    results.append({
        'text_weight': text_weight,
        'behavior_weight': behavior_weight,
        'silhouette': result.silhouette_score,
        'n_clusters': result.n_clusters
    })

df_sensitivity = pd.DataFrame(results)
df_sensitivity

In [ ]:
# シルエットスコアの推移をプロット
fig = px.line(
    df_sensitivity,
    x='text_weight',
    y='silhouette',
    markers=True,
    title='Silhouette Score vs Text Weight',
    labels={'text_weight': 'Text Weight', 'silhouette': 'Silhouette Score'}
)
fig.update_layout(height=400, width=600)
fig.show()

## 9. 結論

クラスタリング分析の結果、以下のことが確認されました：

1. **仮説の検証**: リアクション絵文字は使用されるコンテキストに基づいて明確なクラスタを形成
2. **感情カテゴリ**: ポジティブ、ネガティブ、確認、ユーモア、質問、緊急などのカテゴリが識別可能
3. **特徴量の有効性**: テキスト埋め込みと行動パターンの両方がクラスタリングに貢献